In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as sts

In [8]:
df = pd.read_csv('https://raw.githubusercontent.com/OHrydko/Machine-Learning-Labs/master/lab1/MP-07-Hrydko.csv', 
                 header= None, names=['x0', 'x1', 'x2', 'x3', 'x4', 'x5', 'y'], sep=';',engine='python')
df.head()

,x0,x1,x2,x3,x4,x5,y
0,6248.00,131.3700,260.7370,5229.49,157,25.8162,9.726
1,2114.97,56.1939,97.7716,3601.37,27,39.8544,17.395
2,1164.74,45.0019,63.9749,2026.36,22,42.8624,2.375
3,2192.61,55.3350,93.4638,3336.69,25,38.2677,12.987
4,1168.13,53.4750,63.4258,2812.04,17,39.8232,5.562


In [9]:
train = df.iloc[:200]
# визначення параметрів
system_len = len(train.columns)
alpha = 0.9
# знаходження коефіцієнтів
print(train[train.columns[:-1]].values)
matrix_x = train[train.columns[:-1]].values
solve = np.dot(np.dot(np.linalg.inv(np.dot(np.transpose(matrix_x), matrix_x)), np.transpose(matrix_x)), train['y'].values)

# Оцінка статистичної значимості коефіцієнтів
# Відхилення
q_e = (train['y'] - (np.dot(train[train.columns[:-1]],solve)))
# Оцінка дисперсії
s_2 = (q_e**2).sum()/(len(train) - system_len)
# Коефіцієнти зв'язку
K_coef = np.sqrt(np.diag(s_2 * np.linalg.inv(np.dot(np.transpose(matrix_x), matrix_x))))

t_student = round(sts.t.ppf((1 + alpha)/2, len(train) - system_len), 2)
t_calculated = (np.absolute(solve)/K_coef).round(2)
weight_coef = t_calculated > t_student

# Розрахунок коефіцієнта множинної детермінації
R = round(np.sqrt((np.dot(np.dot(np.transpose(solve), np.transpose(matrix_x)), train['y'].values) -
             len(train) * train['y'].mean())/\
    (np.dot(np.transpose(train['y'].values), train['y'].values) - len(train) * train['y'].mean())), 2)
print('Значення розподілу t = {} при параметрах alpha = {} & p = {} при'.format(t_student, alpha, len(train) - system_len))
for i in range(len(solve)):
    print('b{} = {} значущий {} значення t роподілу = {} '
          .format(i, round(solve[i], 2), weight_coef[i], t_calculated[i]))
print('Коефіцієнт детермінації R = {}, R^2 = {},оцінка дисперції s^2 = {}'.format(R, round(R**2, 2), round(s_2, 2)))

[[6248.      131.37    260.737  5229.49    157.       25.8162]
 [2114.97     56.1939   97.7716 3601.37     27.       39.8544]
 [1164.74     45.0019   63.9749 2026.36     22.       42.8624]
 ...
 [1347.86     46.9127   57.5169 1920.46     17.       43.3073]
 [2208.95     95.5685  107.677  3191.12     32.       36.4987]
 [1652.34     64.1218   86.9748 2844.85     23.       40.32  ]]
Значення розподілу t = 1.65 при параметрах alpha = 0.9 & p = 193 при
b0 = 0.0 значущий True значення t роподілу = 3.21 
b1 = -0.12 значущий True значення t роподілу = 5.4 
b2 = 0.04 значущий False значення t роподілу = 1.53 
b3 = -0.0 значущий True значення t роподілу = 1.71 
b4 = 0.03 значущий True значення t роподілу = 3.77 
b5 = 0.2 значущий True значення t роподілу = 7.45 
Коефіцієнт детермінації R = 0.84, R^2 = 0.71,оцінка дисперції s^2 = 26.58


In [10]:
def q_func(w0, wi):
    return (((w0 + train[train.columns[:-1]] * wi).sum(axis = 1) - train['y'])**2).mean()

# Градієнт функції
def gradient(w0, wi):
    w_0 = (2/len(matrix_x)) * ((w0 + train[train.columns[:-1]] * wi).sum(axis = 1) - train['y']).sum()
    w_i = list((2/len(matrix_x)) * \
    np.dot(np.transpose(matrix_x), ((w0 + train[train.columns[:-1]] * wi).sum(axis = 1) - train['y'])))
    w_i.append(w_0)
    return w_i

def answer(w_arg):
    lam = 0.6
    return w_arg - lam * np.array(gradient(w_arg[-1], w_arg[:-1])) / np.sqrt((np.array(gradient(w_arg[-1], w_arg[:-1]))**2).sum())

In [13]:
w = [np.random.uniform(0, 1, size=system_len)]
epsilon = 0.01
error = []
k = [0]
while True:
    w.append(answer(w[-1]))
    error.append(np.absolute(q_func(w[-1][-1], w[-1][:-1]) - q_func(w[-2][-1], w[-2][:-1])))
    print('Ітерація № = {} Значення помилки = {} \n Значення аргументів {}'.format(k[-1], error[-1], w[-1]))
    if error[-1] < epsilon:
        break
    k.append(k[-1] + 1) 
coef_regretion = w[-1]
print('Ітерацій = {} Помилка = {} \n Вектор значень аргументів {}'.format(k[-1], error[-1], coef_regretion))

Ітерація № = 0 Значення помилки = 7130797.185761136 
 Значення аргументів [-0.05120601  0.37669771  0.05067557 -0.10415655  0.4773332   0.10834242
  0.29932442]
Ітерація № = 1 Значення помилки = 7130216.995440988 
 Значення аргументів [0.32828525 0.38932342 0.06865041 0.35997579 0.48588275 0.1117985
 0.29942995]
Ітерація № = 2 Значення помилки = 7130313.685941909 
 Значення аргументів [-0.0548873   0.37644168  0.05044016 -0.10109921  0.47695532  0.10842616
  0.29932585]
Ітерація № = 3 Значення помилки = 7129875.4215144515 
 Значення аргументів [0.32502167 0.38907393 0.06842689 0.36269056 0.48551138 0.11187596
 0.29943125]
Ітерація № = 4 Значення помилки = 7129948.499539544 
 Значення аргументів [-0.05808033  0.37619329  0.05021864 -0.09844319  0.47658504  0.10850255
  0.29932713]
Ітерація № = 5 Значення помилки = 7129617.177032939 
 Значення аргументів [0.32219091 0.38883122 0.06821567 0.36504898 0.48514675 0.11194688
 0.29943243]
Ітерація № = 6 Значення помилки = 7129672.456852492 
 З

Ітерація № = 60 Значення помилки = 7128752.634948489 
 Значення аргументів [-0.07837946  0.37034765  0.04600001 -0.08115932  0.46734734  0.10959143
  0.29934195]
Ітерація № = 61 Значення помилки = 7128748.71339403 
 Значення аргументів [0.30422655 0.38302423 0.06406491 0.38040322 0.47594938 0.1130001
 0.29944653]
Ітерація № = 62 Значення помилки = 7128749.465334676 
 Значення аргументів [-0.07841785  0.37015114  0.04586993 -0.08111116  0.46703092  0.10961968
  0.29934227]
Ітерація № = 63 Значення помилки = 7128745.578459957 
 Значення аргументів [0.30419415 0.38282792 0.06393507 0.3804464  0.47563323 0.11302825
 0.29944685]
Ітерація № = 64 Значення помилки = 7128746.32447458 
 Значення аргументів [-0.07844922  0.36995486  0.04574013 -0.08106882  0.46671482  0.1096478
  0.29934258]
Ітерація № = 65 Значення помилки = 7128742.464985367 
 Значення аргументів [0.30416799 0.38263182 0.06380549 0.3804844  0.47531739 0.11305628
 0.29944716]
Ітерація № = 66 Значення помилки = 7128743.206291893 

Ітерація № = 120 Значення помилки = 7128661.904789648 
 Значення аргументів [-0.07833421  0.36452283  0.04216818 -0.08071304  0.45796666  0.11041305
  0.29935102]
Ітерація № = 121 Значення помилки = 7128658.252854291 
 Значення аргументів [0.30431648 0.37720311 0.06023646 0.38081212 0.46657479 0.1138206
 0.29945558]
Ітерація № = 122 Значення помилки = 7128658.955937488 
 Значення аргументів [-0.07832031  0.36433071  0.04204211 -0.08070862  0.45765748  0.11043997
  0.29935132]
Ітерація № = 123 Значення помилки = 7128655.3088884195 
 Значення аргументів [0.30433045 0.37701108 0.06011046 0.38081647 0.46626579 0.11384751
 0.29945588]
Ітерація № = 124 Значення помилки = 7128656.011033457 
 Значення аргументів [-0.07830633  0.3641387   0.04191612 -0.08070428  0.45734852  0.11046688
  0.29935161]
Ітерація № = 125 Значення помилки = 7128652.368862703 
 Значення аргументів [0.3043445  0.37681918 0.05998455 0.38082074 0.46595701 0.1138744
 0.29945617]
Ітерація № = 126 Значення помилки = 7128653.

Ітерація № = 179 Значення помилки = 7128574.457507706 
 Значення аргументів [0.30473261 0.37168232 0.05661648 0.380925   0.45770009 0.11459376
 0.29946406]
Ітерація № = 180 Значення помилки = 7128575.133916106 
 Значення аргументів [-0.07790412  0.35881047  0.03842252 -0.08059584  0.44878378  0.11121306
  0.2993598 ]
Ітерація № = 181 Значення помилки = 7128571.625614694 
 Значення аргументів [0.30474698 0.3714937  0.0564929  0.38092872 0.45739723 0.11462017
 0.29946435]
Ітерація № = 182 Значення помилки = 7128572.301121788 
 Значення аргументів [-0.07788975  0.35862187  0.03829894 -0.08059212  0.44848094  0.11123945
  0.29936009]
Ітерація № = 183 Значення помилки = 7128568.797508674 
 Значення аргументів [0.30476134 0.3713052  0.05636939 0.38093244 0.45709457 0.11464655
 0.29946464]
Ітерація № = 184 Значення помилки = 7128569.47211567 
 Значення аргументів [-0.07787539  0.35843338  0.03817545 -0.08058841  0.44817832  0.11126584
  0.29936038]
Ітерація № = 185 Значення помилки = 7128565.

Ітерація № = 242 Значення помилки = 7128489.0561783165 
 Значення аргументів [-0.07746272  0.35301741  0.0346295  -0.08048187  0.43949238  0.11202353
  0.29936869]
Ітерація № = 243 Значення помилки = 7128485.690329054 
 Значення аргументів [0.30518814 0.36570363 0.05270198 0.38104262 0.44811121 0.1154302
 0.29947324]
Ітерація № = 244 Значення помилки = 7128486.338485747 
 Значення аргументів [-0.07744863  0.35283236  0.03450843 -0.08047824  0.43919596  0.1120494
  0.29936897]
Ітерація № = 245 Значення помилки = 7128482.977133801 
 Значення аргументів [0.30520223 0.36551868 0.05258099 0.38104626 0.44781495 0.11545606
 0.29947352]
Ітерація № = 246 Значення помилки = 7128483.624426975 
 Значення аргументів [-0.07743455  0.35264743  0.03438745 -0.0804746   0.43889973  0.11207526
  0.29936925]
Ітерація № = 247 Значення помилки = 7128480.267566322 
 Значення аргументів [0.3052163  0.36533385 0.05246007 0.38104989 0.4475189  0.1154819
 0.29947381]
Ітерація № = 248 Значення помилки = 7128480.9

...

Ітерація № = 847 Значення помилки = 7127811.131707001 
 Значення аргументів [0.3090484  0.31468708 0.01957251 0.38204119 0.36732793 0.12253009
 0.29955102]
Ітерація № = 848 Значення помилки = 7127811.565183292 
 Значення аргументів [-0.07358878  0.30182083  0.00138249 -0.07947978  0.35842165  0.11914856
  0.29944674]
Ітерація № = 849 Значення помилки = 7127809.319919058 
 Значення аргументів [0.30905995 0.31453325 0.01947346 0.38204418 0.36708747 0.12255139
 0.29955125]
Ітерація № = 850 Значення помилки = 7127809.752818781 
 Значення аргументів [-0.07357723  0.30166701  0.00128345 -0.07947678  0.35818122  0.11916986
  0.29944697]
Ітерація № = 851 Значення помилки = 7127807.510550546 
 Значення аргументів [0.3090715  0.31437952 0.01937447 0.38204718 0.36684719 0.12257267
 0.29955148]
Ітерація № = 852 Значення помилки = 7127807.942874468 
 Значення аргументів [-0.07356568  0.30151329  0.00118447 -0.07947379  0.35794096  0.11919114
  0.29944721]
Ітерація № = 853 Значення помилки = 7127805

Ітерація № = 905 Значення помилки = 7127759.560406011 
 Значення аргументів [0.30938045 0.31026366 0.01672648 0.38212732 0.36042186 0.12314215
 0.29955771]
Ітерація № = 906 Значення помилки = 7127759.977470756 
 Значення аргументів [-0.07325676  0.29739783 -0.00146324 -0.07939366  0.35151635  0.11976055
  0.29945344]
Ітерація № = 907 Значення помилки = 7127757.817481501 
 Значення аргументів [0.30939179 0.31011251 0.01662932 0.38213026 0.36018619 0.12316305
 0.29955794]
Ітерація № = 908 Значення помилки = 7127758.233991601 
 Значення аргументів [-0.07324543  0.2972467  -0.00156039 -0.07939072  0.35128071  0.11978145
  0.29945367]
Ітерація № = 909 Значення помилки = 7127756.076884185 
 Значення аргументів [0.30940312 0.30996145 0.01653222 0.3821332  0.35995068 0.12318394
 0.29955817]
Ітерація № = 910 Значення помилки = 7127756.492840378 
 Значення аргументів [-0.0732341   0.29709565 -0.00165748 -0.07938778  0.35104522  0.11980234
  0.29945389]
Ітерація № = 911 Значення помилки = 7127754

Ітерація № = 958 Значення помилки = 7127715.395544652 
 Значення аргументів [-0.0729644   0.29349786 -0.00396843 -0.07931779  0.3454422   0.12029964
  0.29945934]
Ітерація № = 959 Значення помилки = 7127713.309242589 
 Значення аргументів [0.30968399 0.30621452 0.01412552 0.38220608 0.35411557 0.12370185
 0.29956384]
Ітерація № = 960 Значення помилки = 7127713.711589047 
 Значення аргументів [-0.07295326  0.29334908 -0.00406392 -0.0793149   0.34521076  0.1203202
  0.29945956]
Ітерація № = 961 Значення помилки = 7127711.628070186 
 Значення аргументів [0.30969513 0.30606581 0.01403008 0.38220897 0.35388426 0.1237224
 0.29956406]
Ітерація № = 962 Значення помилки = 7127712.029881654 
 Значення аргументів [-0.07294212  0.29320039 -0.00415935 -0.07931201  0.34497947  0.12034074
  0.29945978]
Ітерація № = 963 Значення помилки = 7127709.949142252 
 Значення аргументів [0.30970626 0.3059172  0.01393471 0.38221186 0.35365311 0.12374293
 0.29956429]
Ітерація № = 964 Значення помилки = 7127710.3

Ітерація № = 1021 Значення помилки = 7127662.223193641 
 Значення аргументів [0.31002594 0.30164627 0.01119616 0.38229486 0.34701898 0.12433265
 0.29957074]
Ітерація № = 1022 Значення помилки = 7127662.609283507 
 Значення аргументів [-0.07261134  0.28878128 -0.00699296 -0.07922613  0.33811496  0.12095094
  0.29946646]
Ітерація № = 1023 Значення помилки = 7127660.610220181 
 Значення аргументів [0.31003686 0.30150033 0.01110267 0.38229769 0.34679259 0.12435279
 0.29957096]
Ітерація № = 1024 Значення помилки = 7127660.995796771 
 Значення аргументів [-0.07260043  0.28863535 -0.00708645 -0.0792233   0.33788859  0.12097107
  0.29946668]
Ітерація № = 1025 Значення помилки = 7127658.999399859 
 Значення аргументів [0.31004777 0.30135447 0.01100924 0.38230053 0.34656635 0.12437292
 0.29957118]
Ітерація № = 1026 Значення помилки = 7127659.384463861 
 Значення аргументів [-0.07258952  0.28848951 -0.00717987 -0.07922047  0.33766238  0.1209912
  0.2994669 ]
Ітерація № = 1027 Значення помилки = 7

KeyboardInterrupt: 

In [ ]:
test = df.iloc[200:]
error_2_mnk = (test['y'] - (np.dot(test[test.columns[:-1]],solve)))
print(test[test.columns[:-1]])
error_2_grad = (test['y'] - (coef_regretion[-1] + test[test.columns[:-1]] * coef_regretion[:-1]).sum(axis = 1))
print('Похибка МНК = {} \nПохибка градієнта = {}'.format(error_2_mnk, error_2_grad))
    
error_2_mnk = ((test['y'] - (np.dot(test[test.columns[:-1]],solve)))**2).mean()
error_2_grad = ((test['y'] - (coef_regretion[-1] + test[test.columns[:-1]] * coef_regretion[:-1]).sum(axis = 1))**2).mean()
print('Похибка МНК = {} \nПохибка градієнта = {}'.format(error_2_mnk, error_2_grad))